## Cleaning

In [22]:
import pandas as pd

In [27]:
df = pd.read_csv("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/selected_samples_osint_mmsd.csv")

In [29]:
df.head(20)

,image_id,text,subreddit,major_label
0,1mau6q6,look into my eyes.. and give treats 👀,aww,0
1,h8vdb8,lol sex and weed number funny,FellowKids,1
2,m5xe5w,kenyon martin called out jeremy lin for his ha...,popular,0
3,5wg0gy,i can see it coming,sarcasm,1
4,1mbsoge,this right here? peak love 😍,popular,0
5,qzejm0,why stop at just one stick up the ass when you...,technicallythetruth,1
6,1ith0ez,tv bad,im14andthisisdeep,1
7,lxiydt,throwback to when my goat was standing on my d...,AnimalsBeingJerks,0
8,1mbpyzr,meirl,popular,1
9,kylmnj,found a patch of moss behind a loading dock th...,mildlyinteresting,0


### Replace username and subreddits with tokens. And remove emojis

In [30]:
import re
import emoji

for idx, row in df.iterrows():
    text = row["text"]
    user_cleaned_text = re.sub(r'/?u/\S+', '<user>', text)
    subreddit_cleaned_text = re.sub(r'/?r/\S+', '<subreddit>', user_cleaned_text)
    emoji_removed = emoji.replace_emoji(subreddit_cleaned_text, replace='')
    df.loc[idx, "text"] = emoji_removed

In [31]:
df.head()

,image_id,text,subreddit,major_label
0,1mau6q6,look into my eyes.. and give treats,aww,0
1,h8vdb8,lol sex and weed number funny,FellowKids,1
2,m5xe5w,kenyon martin called out jeremy lin for his ha...,popular,0
3,5wg0gy,i can see it coming,sarcasm,1
4,1mbsoge,this right here? peak love,popular,0


# Splitting into train, valid and test

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/selected_samples_osint_mmsd_cleaned.csv")

In [2]:
df.shape

(23288, 4)

In [3]:
df_pos = df[df["label"] == 1]
df_neg = df[df["label"] == 0]

In [12]:
pos_splits = {}
neg_splits = {}

In [13]:
for key, group in df_pos.groupby("subreddit"):
    train, test = train_test_split(group, test_size=0.2, random_state=42)
    valid, test = train_test_split(test, test_size=0.5, random_state=42)
    pos_splits[key] = [train.reset_index(drop = True), valid.reset_index(drop=True), test.reset_index(drop=True)]

In [8]:
df_neg["subreddit"].value_counts(sort=False)

subreddit
aww                      842
popular                 1054
ATBGE                    642
mildlyinteresting       2063
notinteresting           735
popculturechat           313
Damnthatsinteresting     667
technicallythetruth      130
PrequelMemes             177
pics                    1120
FellowKids               285
mildlyinfuriating        631
BeAmazed                 382
funny                    175
AnimalsBeingJerks        144
memes                     47
im14andthisisdeep        178
madlads                   77
FUCKYOUINPARTICULAR      124
PeopleFuckingDying       192
sports                   114
trippingthroughtime       62
rareinsults               61
sarcasm                   68
qualityrareinsults         5
Name: count, dtype: int64

In [16]:
for key, group in df_neg.groupby("subreddit"):
    train, test = train_test_split(group, test_size=0.2, random_state=45)
    valid, test = train_test_split(test, test_size=0.5, random_state=45)
    neg_splits[key] = [train.reset_index(drop = True), valid.reset_index(drop=True), test.reset_index(drop=True)]

ValueError: With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [55]:
train = None
valid = None
test = None
for key, value in splits.items():
    train = pd.concat([train, value[0]])
    valid = pd.concat([valid, value[1]])
    test = pd.concat([test, value[2]])
print(train.shape)
print(valid.shape)
print(test.shape)

(18617, 4)
(2330, 4)
(2341, 4)


In [63]:
train.to_csv("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/train.csv", index=False)
valid.to_csv("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/valid.csv", index=False)
test.to_csv("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/test.csv", index=False)

In [64]:
train.to_json("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/train.json", orient="records", lines=False)
valid.to_json("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/valid.json", orient="records", lines=False)
test.to_json("/Users/sinngamkhaidem/Developer/mllm-based-mmsd-osint/mllm_finetune/gpt_predictions/test.json", orient="records", lines=False)